In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import pandas as pd
import feather
import os
from six.moves import range

D:\Programming\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.enable_eager_execution()

print("TensorFlow version: {}".format(tf.VERSION))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 1.11.0
Eager execution: True


In [3]:
data_root ='D:\\Programming\\Projects\\notMNIST'
image_size=28

def feather_3d(file,image_size = 28):
    temp = feather.read_dataframe(file)
    width = np.shape(temp.values)[1]
    if width == 1:
        final = temp.values
        final = final.reshape(-1)
    else:
        final = np.reshape(temp.values, (-1, image_size, image_size))
    return final

def read_data(dir, filename):
    file = os.path.join(dir, filename + '.feather')
    output = feather_3d(file)
    return output

test_dataset = read_data(data_root, 'test_dataset_final')
test_labels = read_data(data_root, 'test_labels_final')
train_dataset = read_data(data_root, 'train_dataset_final')
train_labels = read_data(data_root, 'train_labels_final')
valid_dataset = read_data(data_root, 'valid_dataset_final')
valid_labels = read_data(data_root, 'valid_labels_final')

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 28, 28) (197309,)
Validation set (17460, 28, 28) (17460,)
Test set (10202, 28, 28) (10202,)


In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (197309, 784) (197309, 10)
Validation set (17460, 784) (17460, 10)
Test set (10202, 784) (10202, 10)


In [11]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():
    
  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random values following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [12]:
num_steps = 801



with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 16.517284
Training accuracy: 9.4%
Validation accuracy: 13.7%
Loss at step 100: 2.383195
Training accuracy: 71.0%
Validation accuracy: 69.0%
Loss at step 200: 1.920660
Training accuracy: 74.2%
Validation accuracy: 71.6%
Loss at step 300: 1.664920
Training accuracy: 75.7%
Validation accuracy: 72.6%
Loss at step 400: 1.492354
Training accuracy: 76.7%
Validation accuracy: 72.9%
Loss at step 500: 1.365133
Training accuracy: 77.4%
Validation accuracy: 73.3%
Loss at step 600: 1.266274
Training accuracy: 77.8%
Validation accuracy: 73.3%
Loss at step 700: 1.186686
Training accuracy: 78.4%
Validation accuracy: 73.4%
Loss at step 800: 1.120736
Training accuracy: 78.8%
Validation accuracy: 73.5%
Test accuracy: 81.5%


In [13]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [14]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 16.848394
Minibatch accuracy: 9.4%
Validation accuracy: 12.8%
Minibatch loss at step 500: 1.508445
Minibatch accuracy: 76.6%
Validation accuracy: 74.0%
Minibatch loss at step 1000: 1.594661
Minibatch accuracy: 75.8%
Validation accuracy: 74.7%
Minibatch loss at step 1500: 1.428557
Minibatch accuracy: 72.7%
Validation accuracy: 75.7%
Minibatch loss at step 2000: 0.917899
Minibatch accuracy: 77.3%
Validation accuracy: 76.4%
Minibatch loss at step 2500: 1.020791
Minibatch accuracy: 79.7%
Validation accuracy: 76.6%
Minibatch loss at step 3000: 1.061759
Minibatch accuracy: 81.2%
Validation accuracy: 77.3%
Test accuracy: 85.1%


In [6]:
batch_size = 128


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
    
  x = tf.matmul(tf_train_dataset, weights1) + biases1
  x = tf.nn.relu(x)
  logits = tf.matmul(x, weights2) + biases2
  # Training computation.
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  y = tf.matmul(tf_valid_dataset, weights1) + biases1
  y = tf.nn.relu(y)
  logits_valid = tf.matmul(y, weights2) + biases2
  valid_prediction = tf.nn.softmax(logits_valid)

  z = tf.matmul(tf_test_dataset, weights1) + biases1
  z = tf.nn.relu(z)
  logits_test = tf.matmul(z, weights2) + biases2
  test_prediction = tf.nn.softmax(logits_test)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [7]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 258.349335
Minibatch accuracy: 12.5%
Validation accuracy: 26.5%
Minibatch loss at step 500: 13.973191
Minibatch accuracy: 82.8%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 17.270746
Minibatch accuracy: 75.8%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 8.468081
Minibatch accuracy: 78.1%
Validation accuracy: 78.9%
Minibatch loss at step 2000: 3.647547
Minibatch accuracy: 82.8%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 6.976220
Minibatch accuracy: 79.7%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 4.743351
Minibatch accuracy: 85.2%
Validation accuracy: 80.0%
Minibatch loss at step 3500: 4.195789
Minibatch accuracy: 81.2%
Validation accuracy: 80.2%
Minibatch loss at step 4000: 1.561752
Minibatch accuracy: 84.4%
Validation accuracy: 79.9%
Minibatch loss at step 4500: 2.748359
Minibatch accuracy: 81.2%
Validation accuracy: 81.2%
Minibatch loss at step 5000: 2.314289
Minibatch accuracy: 84.4%
Validation acc